# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. Using DeepLabCut for 3D markerless pose estimation during behavior across species.

pre-print: https://www.biorxiv.org/content/10.1101/476531v1

In [430]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np

wd=r"E:\Users\Phil\DeepLabCut\dev"
vid_directory=r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run5flexextLAR_90-35-4ms"
cam_prefix = "robo_CLAHE_CameraNo."

In [2]:
markerlist = ['Body_ds1_crn',
              'Body_ds2_int',
              'Body_ds3_cdl',
              'Body_vn1_crn',
              'Body_vn2_int',
              'Body_vn3_cdl',
              'Body_acc',
              'Scapula_acr',
              'Scapula_spi',
              'Scapula_vtb',
              'Scapula_acc',
              'Humerus_dpc',
              'Humerus_ent',
              'Humerus_ect',
              'Humerus_acc',
              'Ulna_olc',
              'Ulna_int',
              'Ulna_dst',
              'Ulna_acc',
              'Radius_prx',
              'Radius_int',
              'Radius_dst',
              'Radius_acc',
              'Teres_maj_prx',
              'Teres_maj_dst',
              'Biceps_prx',
              'Biceps_dst']
dotsize = 5
corner2move2 = 512

In [362]:
# path_config_file=r"E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\config.yaml"

In [365]:
task='23Apr18CLAHE'
experimenter='Phil'
video=[vid_directory+r'\robo_CLAHE_CameraNo.1.avi',
       vid_directory+r'\robo_CLAHE_CameraNo.2.avi'] # Enter the paths of your videos you want to grab frames from.
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory=wd,copy_videos=False) 

Created "E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\videos"
Created "E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\labeled-data"
Created "E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\training-datasets"
Created "E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\dlc-models"
Creating the symbolic link of the video
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run5flexextLAR_90-35-4ms\robo_CLAHE_CameraNo.1.avi to E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\videos\robo_CLAHE_CameraNo.1.avi
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run5flexextLAR_90-35-4ms\robo_CLAHE_CameraNo.2.avi to E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\videos\robo_CLAHE_CameraNo.2.avi
E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\videos\robo_CLAHE_CameraNo.1.avi
E:\Users\Phil\DeepLabCut\dev

In [366]:
%%capture
## substitute markers from 23apr18 experiment ##
config = ruamel.yaml.load(open(path_config_file))
config['bodyparts']=markerlist
config['dotsize']=dotsize
config['corner2move2']=[corner2move2,corner2move2]
ruamel.yaml.round_trip_dump(config, sys.stdout)
with open(path_config_file, 'w') as fp:
    ruamel.yaml.round_trip_dump(config, fp)
    fp.close()

In [367]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:05, 140.54it/s]


Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:06, 125.12it/s]


Kmeans clustering ... (this might take a while)

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [462]:
#### convert xmalab exports to training format ####
# no spaces in marker names, otherwise 2D export fails (more header columns than data)
##!!!! CHECK FOR FLIPPING!!!!
xma_2Dcsv_path = r"E:\Users\Phil\XROMM\23Apr18\run5flexextLAR_90-35-4ms\2D_run5flexextLAR_90-35-4ms.csv"
labeled_data_path=path_config_file.split('config.yaml')[0]+"labeled-data\\"

## import XMAlab 2D exports
df = pd.read_csv(xma_2Dcsv_path, sep=',', header=0)

## coerce data into DeepLabCut hierarchical format
df['frame_index']=df.index
df['scorer']=experimenter
df = df.melt(id_vars=['frame_index','scorer'])
new = df['variable'].str.rsplit("_",n=2,expand=True)
df['variable'],df['cam'],df['coords'] = new[0], new[1], new[2]
df=df.rename(columns={'variable':'bodyparts'})
df['coords']=df['coords'].str.rstrip(" ").str.lower()
df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype("category")
df['bodyparts'].cat.set_categories(markerlist,inplace=True)
df['frame_index'] = ['labeled-data\\' + cam_prefix+x[-1] + '\\img' + (f"{y:03d}") + '.png' for x, y in zip(df['cam'], df['frame_index'])]
newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first')
newdf.index.name=None
newdf = newdf.dropna()
newdf = newdf.astype(np.float64)

## go into frame folders and get frame index ##
extracted_frames = []
for root, dirs, files in os.walk(os.path.join(os.path.dirname(path_config_file),'labeled-data')):
    for name in files:
        if name.endswith(".png"):
            camera_id = root.split(' ')[-1][-1]
            frame_no = int(name.split('.')[0].replace('img',''))
            new_name = 'labeled-data\\'+cam_prefix+camera_id+'\\img' + (f"{frame_no:03d}") + '.png'
            extracted_frames.append(new_name)
        
## filter by list of extracted frames
df_extracted = newdf.filter(items=pd.Index(extracted_frames),axis=0)

## split new df into cams 1 and 2
df1 = df_extracted.filter(like=cam_prefix+"1",axis=0)
df2 = df_extracted.filter(like=cam_prefix+"2",axis=0)

## split new df into cams 1 and 2, export as h5 and csv
for x in [1,2]:
    cam_name = cam_prefix+str(x)
    dfx = df_extracted.filter(like=cam_name,axis=0)
    data_name = labeled_data_path+cam_name+"\\CollectedData_"+experimenter+".h5"
    dfx.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
    dfx.to_csv(data_name.split('.h5')[0]+'.csv')


E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\labeled-data\robo_CLAHE_CameraNo.1\CollectedData_Phil.h5
E:\Users\Phil\DeepLabCut\dev\23Apr18CLAHE-Phil-2019-09-24\labeled-data\robo_CLAHE_CameraNo.2\CollectedData_Phil.h5


In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

In [5]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\labeled-data\Camera No.1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\labeled-data\Camera No.2_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [16]:
deeplabcut.create_training_dataset(path_config_file)

E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\training-datasets\iteration-0\UnaugmentedDataSet_23Apr18Sep20  already exists!
E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\dlc-models\iteration-0\23Apr18Sep20-trainset95shuffle1  already exists!
E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\dlc-models\iteration-0\23Apr18Sep20-trainset95shuffle1//train  already exists!
E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\dlc-models\iteration-0\23Apr18Sep20-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file, displayiters=10,saveiters=100)

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

In [42]:
videofile_path = [r'Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run5flexextLAR_90-35-4ms'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Using snapshot-1030000 for model E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\dlc-models\iteration-0\23Apr18Sep20-trainset95shuffle1
num_outputs =  1
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\23Apr18-Phil-2019-09-20\dlc-models\iteration-0\23Apr18Sep20-trainset95shuffle1\train\snapshot-1030000
Analyzing all the videos in the directory
Starting to analyze %  Camera No.2.avi
Loading  Camera No.2.avi
Duration of video [s]:  26.67 , recorded with  30.0 fps!
Overall # of frames:  800  found with (before cropping) frame dimensions:  1024 1024
Starting to extract posture


810it [01:14, 11.05it/s]                                                                                               

Detected frames:  800


810it [01:14, 10.84it/s]


...

810it [00:47, 15.43it/s]                                                                                               

Detected frames:  800


810it [00:47, 17.17it/s]


Saving results in ....
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_23Apr18Sep20shuffle1_1030000'

In [43]:
deeplabcut.create_labeled_video(path_config_file,[r'Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run5flexextLAR_90-35-4ms'])

Analyzing all the videos in the directory
Starting %  . ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run5flexextLAR_90-35-4ms']
Loading  Camera No.1.avi and data.
False 0 1024 0 1024
800
Duration of video [s]:  26.67 , recorded with  30.0 fps!
Overall # of frames:  800 with cropped frame dimensions:  1024 1024
Generating frames and creating video.


100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 60.22it/s]


Starting %  . ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run5flexextLAR_90-35-4ms']
Loading  Camera No.2.avi and data.
False 0 1024 0 1024
800
Duration of video [s]:  26.67 , recorded with  30.0 fps!
Overall # of frames:  800 with cropped frame dimensions:  1024 1024
Generating frames and creating video.


100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 59.72it/s]


## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [ ]:
deeplabcut.extract_outlier_frames?

In [40]:
deeplabcut.create_labeled_video?

Signature: deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)
Docstring:
    Labels the bodyparts in a video. Make sure the video is already analyzed by the function 'analyze_video'

    Parameters
    ----------
    config : string
        Full path of the config.yaml file as a string.

    videos : list
        A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.
    
    videotype: string, optional
        Checks for the extension of the video in case the input to the video is a directory.
 Only videos with this extension are analyzed. The default is ``.avi``

    shuffle : int, optional
        Number of shuffles of training dataset. Default is

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)